# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
It is a classification problem. We want to distinguish students that are likely to fail and who supposedly need intervention from those students who don't. Consequently, we have a discrete output variable with two labels -- a job for classification.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
'''
# get an overview first
print student_data.head(), '\n'
print student_data.describe(), '\n'
'''

# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
# minus 1 because the last column represents the target variable
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = student_data['passed'][student_data['passed'] == 'yes'].count()

# TODO: Calculate failing students
# don't use (n_students - n_passed) because there might be missing data
n_failed = student_data['passed'][student_data['passed'] == 'no'].count()

# TODO: Calculate graduation rate
grad_rate = float(n_passed) / n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)

print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn import cross_validation

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above

# random_state, also for later
RANDOM_STATE = 31415

# test_size is 1-train_size by default, so no need to use num_test
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, train_size=num_train, random_state=RANDOM_STATE)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

## The problem at a glance
We have a pretty small dataset with only 395 rows, and we will have to shrink it even more for creating a test set. In relation to the number of rows, we have quite a lot of features (30). In consequence, we will have to use a classification algorithm that can build a feasible model even given few training items and relatively many features.

Furthermore, we could assume that the school might collect data for more students in the future, and this would influence our choice for the algorithm as well. It needs to scale well CPU-wise if we use a parametric model and want to retrain, or memory-wise if we use a non-parametric model. Since there are no hints given in the assignment -- and the board of directors probably wants to see some numbers right now, we won't take this into account too much, but keep it in mind.

### Classifier 1: Support Vector Machines
#### real-world application
Support Vector Machines seem to be used frequently in medical settings. For example, they're used to identify breast cancer ([Zheng, B., Yoon, S. W., & Lam, S. S. (2014). Breast cancer diagnosis based on feature extraction using a hybrid of K-means and support vector machine algorithms. Expert Systems with Applications, 41(4), 1476-1482.](http://sci-hub.cc/10.1016/j.eswa.2013.08.044)). In the paper mentioned, SVMs are used in combination with a K-means algorithm. Using a 10-fold cross validation, this approach has 97.38% accuracy when tested on the [Wisconsin Diagnostic Breast Cancer (WDBC) data set](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)).
SVMs are also used on MRI images to distinguish between Alzheimer’s disease, mild cognitive impairment, and normal controls ([Zhang, Y., Wang, S., & Dong, Z. (2014). Classification of Alzheimer disease based on structural magnetic resonance imaging by kernel support vector machine decision tree. Progress In Electromagnetics Research, 144, 171-184.](http://jpier.org/PIER/pier144/16.13121310.pdf)). After all, this approach achieves 80% classification accuracy.

#### strengths
SVMs can be very effective in in high dimensional spaces. In fact, they can still be effective if the number of features is greater than the number of samples. Also, they can be used for many purposes: we can use the kernel trick to create non-linear decision boundaries in order to capture very complex patterns in the data. (cmp. [sklearn tutorial](http://scikit-learn.org/stable/modules/svm.html#svm))

#### weaknesses
Unfortunately, as we have learned from Sebastian Thrun, SVMs don't perform well on large training data sets because of the rapid growth of computation time. Also, they are prone to overfitting if there's a lot of noise in the data. Finetuning this via different parameters can be tricky.

#### justification
Using a support vector machine might be a good idea, because we only have a small dataset of a non-critical size. The [API page of sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) reads "hard to scale to dataset with more than a couple of 10000 samples", and we are way below that. Also taking into account the relatively high number of features (compared to the sample size), a SVM might be benefitial. It should be able to create a feasible model given these circumstances.


### Classifier 2: Logistic Regression
#### real-world application
Logistic regression, despite its name, is a linear model for classification rather than regression (cmp. [sklearn documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)). The algorithm is used in all kinds of domains ([Chatterjee, S., & Hadi, A. S. (2015). Regression analysis by example. John Wiley & Sons.](https://books.google.de/books?hl=de&lr=&id=zyjWBgAAQBAJ&oi=fnd&pg=PP1&dq=logistic+regression+example&ots=OYhEeTZMlZ&sig=x14Ux9ZP_gHEKsqmJP-expZFYHc#v=onepage&q=logistic%20regression%20example&f=false)) and can e.g. be used for classifying human actions in videos ([Liu, W., Liu, H., Tao, D., Wang, Y., & Lu, K. (2015). Multiview Hessian regularized logistic regression for action recognition. Signal Processing, 110, 101-107.](http://sci-hub.cc/10.1016/j.sigpro.2014.08.002)). This approach can support automatic tagging of videos, and in consequence improve information retrieval.

#### strengths
Logistic regression is a rather simple approach that does not require much computation time, so it can build a model quickly. In consequence, it can be used on large data sets. Furthermore, it is easy to interpret the structure of the model, so you could analyze it to understand the underlying problem better.

#### weaknesses
The simplicity of the model is also a weakness. Logistic regression assumes that data can be separated by a smooth linear decision boundary. If that assumption is wrong, then the algorithm will not yield good results. Also, logistic regression needs enough data points that cover the sets of features that can explain the target variable. If there are important factors that can influence the result, but only few data points reflect this factor, the explanatory power of the model is reduced (cmp [What are the advantages of logistic regression over decision trees?](https://www.quora.com/What-are-the-advantages-of-logistic-regression-over-decision-trees)).

#### justification
On first sight, logistic regression might be a weird choice. We don't know if the data can be separated in a linear fashion. On the other hand, linear models can work well in high dimensional spaces depending on the data structure (cmp. [sklearn classifier comparison](http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)). Also, we have a binary classification problem (*student passed* or *student didn't pass*), and logistic regression is pretty good for one of those. That is why I'd like to give it a shot.

### Classifier 3: K-Nearest Neighbors
#### real-world application
The K-Nearest Neighbor algorithm has already been used e.g. for estimating forest aboveground biomass in Norwegian forests [McRoberts, R. E., Næsset, E., & Gobakken, T. (2015). Optimizing the k-Nearest Neighbors technique for estimating forest aboveground biomass using airborne laser scanning data. Remote Sensing of Environment, 163, 13-22.](http://sci-hub.cc/10.1016/j.rse.2015.02.026)).

#### strengths
One of the benefits of nearest neighbor classifiers is that it doesn't need real training. This makes it suitable for applications that can or need to update the data frequently as new information is collected. Additionally, nearest neighbor algorithms can handle noise very well, and you don't have to analyze the data for possible patterns beforehand.

#### weaknesses
On the downside, nearest neighbor classifiers don't derive a parametric model but store all the datapoints. This can require lots of memory for large data sets. Furthermore, they are slow when actually predicting a result.

#### justification
"Being a non-parametric method, it is often successful in classification situations where the decision boundary is very irregular." ([sklearn documentation](http://scikit-learn.org/stable/modules/neighbors.html)). Since we didn't analyze the data beforehand and we don't know, K-Nearest Neighbors might be a good approach. Also, I think it suits the organizational requirements, because you will get new data every now and then that would need re-training the model. Furthermore, you probably won't need predictions on a daily basis, so the lower speed for predictions won't be a problem.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = SVC(random_state = RANDOM_STATE)
clf_B = LogisticRegression(random_state = RANDOM_STATE)
clf_C = KNeighborsClassifier()

clfs = [clf_A, clf_B, clf_C]

# TODO: Set up the training set sizes

# test sample sizes
sample_sizes = [100, 200, 300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

# I made this a little more compact and flexible
for clf_item in clfs:
    for sample_size in sample_sizes:
        print '\nclf:', clf_item.__class__.__name__, '/ size:', sample_size, '\n'
        train_predict(clf_item, X_all[:sample_size], y_all[:sample_size], X_test, y_test)
    print '============================================================'


clf: SVC / size: 100 

Training a SVC using a training set size of 100. . .
Trained model in 0.0017 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8772.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.8000.

clf: SVC / size: 200 

Training a SVC using a training set size of 200. . .
Trained model in 0.0041 seconds
Made predictions in 0.0031 seconds.
F1 score for training set: 0.8474.
Made predictions in 0.0019 seconds.
F1 score for test set: 0.8133.

clf: SVC / size: 300 

Training a SVC using a training set size of 300. . .
Trained model in 0.0095 seconds
Made predictions in 0.0068 seconds.
F1 score for training set: 0.8686.
Made predictions in 0.0022 seconds.
F1 score for test set: 0.8633.

clf: LogisticRegression / size: 100 

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0013 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.9200.
Made predictions in 0.0001 seconds.
F1 score for 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Support Vector Machine (SVM) **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0017 seconds      |      0.0008 seconds    |      0.8772      |       0.8000    |
| 200               |     0.0041 seconds      |      0.0019 seconds    |      0.8474      |       0.8133    |
| 300               |     0.0095 seconds      |      0.0022 seconds    |      0.8686      |       0.8633    |

** Classifer 2 - Logistic Regression **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0013 seconds     |       0.0001 seconds   |      0.9200      |       0.7313    |
| 200               |      0.0023 seconds     |       0.0002 seconds   |      0.8414      |       0.8281    |
| 300               |      0.0029 seconds     |       0.0002 seconds   |      0.8486      |       0.8750    |

** Classifer 3 - K-Nearest-Neighbors **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0006 seconds     |       0.0014 seconds   |      0.8750      |      0.7518     |
| 200               |      0.0006 seconds     |       0.0021 seconds   |      0.8600      |      0.8058     |
| 300               |      0.0009 seconds     |       0.0030 seconds   |      0.8597      |       0.8421    |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

I suggest to use the SVM model.

While on first sight the Logistic Regression might promise the best results with a steady increase of the F1 score for the test data up to 0.875, the latter is higher than the corresponding F1 score for the training data. It seems that the algorithm is still "working" on finding the best model, probably because the sample size is still too small to build a robust model using logistic regression.

Both, K-Nearest Neighbors and SVM yield have improved and return high F1 scores that are close together -- thus don't seem to suffer from high bias or high variance -- so the models seem to be fine. The tests also confirm my statement about time for training and prediction. The SVM gives slightly better results, so without being able to ask for more information about the performance of the resources and the use case, I'd go for it.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Think of the planets belonging to the Federation and the Romulan Empire in Star Trek. Both partys don't really like each other and have a demilitarized zone in between their territories with the border right in the middle. Certainly, they want it to keep each other as far away as possible, so they want the demilitarized zone to be as wide as possible -- someone traveling in the wrong direction should not accidentally enter the other party's territory.

Now think of the students with different features as different planets. We want to draw a line between those students (planets) who will pass the test (Romulan Empire) and those who will fail (Federation). This line will run right in the middle of the demilitarized zone of maximum width. This way, you will most likely neither miss a student that might intervention nor bother someone who doesn't. That's what the SVM does.

If you are traveling in space between both territories and you want to know whether your vessel is in Federation territory or in Romulan territory, you just have to find your location on a map and see on which side of the line you are. Quick and easy. That's what the SVM does for predicting if a student will fail or not.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [8]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

# TODO: Create the parameters list you wish to tune
parameters = {
    'kernel':['linear', 'rbf'],
    'C':[0.1, 0.3, 0.7, 1.0, 1.3, 1.7],
    'gamma': [0.02, 0.03, 0.033, 0.04]
}

# TODO: Initialize the classifier
clf = clf_A

# Let's have a look at an untuned SVM
clf.fit(X_train, y_train)
print "Untuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print 'Untuned model has a testing F1 score of {:.4f}'.format(predict_labels(clf, X_test, y_test)), '\n'

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

## What's the result?
print'\nBest parameters:'
print grid_obj.best_params_, '\n'


Made predictions in 0.0073 seconds.
Untuned model has a training F1 score of 0.8541.
Made predictions in 0.0025 seconds.
Untuned model has a testing F1 score of 0.8133 

Made predictions in 0.0074 seconds.
Tuned model has a training F1 score of 0.8620.
Made predictions in 0.0025 seconds.
Tuned model has a testing F1 score of 0.8158.

Best parameters:
{'kernel': 'rbf', 'C': 0.7, 'gamma': 0.04} 



### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

For the fine tuned model, I chose some parameters close to sklearn's default values that were used to the untuned model and gave us the F1 score of 0.8541 for training and 0.8133 for testing.

For the tuned model, the final F1 score for training is 0.8620, and the final F1 score for testing is 0.8158. It seems that we were able to increase both F1 scores a little bit - especially the one for testing, but we also increased the variance. We might probably tweak this a little more, but we should be careful not to overfit the model.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.